In [1]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer, *-*"

#r "nuget: Azure.AI.OpenAI"
#r "nuget: Azure.Identity"
#r "nuget: Azure"
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Data.SqlClient"

Loading extension script from `C:\Users\hasan\.nuget\packages\microsoft.dotnet.interactive.sqlserver\1.0.0-beta.25177.1\interactive-extensions\dotnet\extension.dib`

Query Microsoft SQL Server databases. 
 This extension adds support for connecting to Microsoft SQL Server databases using the #!connect mssql magic command. For more information, run a cell using the #!sql magic command.

In [2]:
#!connect mssql --kernel-name Stackoverflow --connection-string "Server=tcp:hsavransql1.database.windows.net,1433;Initial Catalog=azuredb1;Persist Security Info=False;User ID=hsavran;Password=HasanSavran90;MultipleActiveResultSets=False;Encrypt=True;TrustServerCertificate=False;Connection Timeout=30;""

Kernel added: #!sql-Stackoverflow

In [ ]:
ALTER TABLE Posts ADD BodyVector VECTOR(1536)

In [4]:
SELECT TOP 1 * FROM Posts WHERE PostType = 'Question' and BodyVector IS not NULL

(1 row affected)

PostId Title PostBody ViewCount PostType OwnerUserId Score BodyVector 15228 Setting an ASP.NET Master Page at runtime <p>I'm working on a site which needs to be able to support two or more looks, changable at runtime. I'd hoped to be able to handle the change with a CSS switch, but it looks like I'll need to use a different masterpage for each design.</p>

<p>So, what's the best way to set the masterpage at runtime? Page.MasterPageFile can only be set in the Page.OnPreInit event. It looks like the solutions are to make all my pages inherit from a common base which handles the PreInit event, or to use an HttpModule which does that.</p>

<p>Any advice?</p>
 8529 Question 5 8 [-1.6122393e-003,2.9044395e-002,-2.9285285e-003,-2.1514878e-002,-6.2872167e-003,1.8899504e-002,-8.9610908e-003,-1.3544874e-002,-2.6676796e-002,-1.4742440e-002,2.6718091e-002,-1.1528285e-002,2.8080838e-002,1.3758234e-002,1.3476049e-002,1.6091421e-002,2.2671146e-002,6.1323592e-003,-8.9129130e-004,-4.8934990e-003,4.3807481e-003,1.6913887e-003,1.9229868e-002,-3.8377143e-002,-4.0331792e-002,-1.1789822e-002,1.1631523e-003,-1.7013684e-002,-1.0358250e-002,-1.5829884e-002,3.2926157e-002,-8.4380163e-003,-2.1886535e-002,-3.4027368e-002,-2.9815242e-002,1.6531905e-002,1.0846912e-002,-1.9463874e-002,-2.3194221e-003,-1.9450109e-002,8.7064365e-003,-1.2278484e-002,1.6669556e-002,-1.0268777e-002,-2.3648471e-002,1.7247690e-002,4.1919082e-004,-2.6539145e-002,-2.7984481e-002,3.5541531e-002,2.3772355e-002,2.2106776e-002,2.5551496e-003,-3.6339909e-002,-1.4990211e-002,-2.4707308e-005,-2.1156983e-002,3.5458941e-002,1.1528285e-002,6.5298268e-004,4.1543122e-002,2.8603911e-002,-6.6726403e-003,1.9573996e-002,-5.9293238e-003,-1.0131125e-002,-3.6587678e-002,1.8651733e-002,-1.9133511e-003,-3.5706710e-002,4.5424882e-002,3.1935070e-002,-3.4440320e-002,-2.5396639e-002,1.7054979e-002,-1.5843648e-002,-2.2203133e-002,-3.7138283e-002,5.0724456e-003,-4.0469444e-003,5.7916725e-003,-3.2210373e-003,-3.4688093e-002,2.6869508e-002,1.5471991e-002,-6.0007302e-004,3.6670271e-002,1.0633552e-002,-1.8995861e-003,-1.0661082e-002,1.7371576e-002,4.2148788e-002,3.1384464e-002,1.5155393e-002,5.5662688e-004,-1.0076066e-002,-4.6354029e-003,-1.9718530e-003,-1.5588994e-003,-2.1294635e-002,-1.2698320e-003,1.5389401e-002,-3.4605503e-002,-1.0337602e-002,-2.8190957e-002,1.0461489e-002,-3.9295104e-004,5.2032140e-003,8.5825501e-003,8.1214187e-004,-4.1350409e-002,4.0992517e-002,2.0922977e-002,-2.0523788e-002,-8.3554257e-003,-1.5843648e-002,-1.5637172e-002,-3.5293758e-002,-8.9817382e-003,2.3841182e-002,1.4444769e-003,3.0035485e-002,9.2226276e-003,-1.2037594e-002,3.4219219e-004,-5.4441034e-003,-4.0579565e-002,7.3918677e-003,1.3634347e-002,3.5858129e-003,7.1688727e-002,4.3222466e-003,1.1039623e-002,1.3861472e-002,-3.5871893e-002,1.2650141e-002,-3.7193343e-002,-5.7159644e-003,-1.9684117e-002,-7.8942943e-003,2.5782064e-002,1.5664702e-002,1.4246895e-002,1.0502784e-002,1.5293044e-002,1.8527847e-002,6.6279033e-003,-1.2140833e-002,-1.0908855e-002,-2.3868712e-002,-3.6477558e-003,-1.0915738e-002,1.1314926e-002,-6.0394448e-003,5.0352797e-002,7.0477393e-003,-4.0882397e-003,1.0385781e-002,-2.6411817e-003,-8.1764795e-003,1.4687379e-002,2.7626589e-002,1.2767145e-002,3.1329405e-002,-2.4020128e-002,3.1219283e-002,7.8323511e-003,-1.9450109e-002,-1.1762292e-002,6.7621134e-003,-2.3469523e-002,5.2410681e-003,-3.4853276e-002,-1.1782940e-002,-1.5513286e-002,1.7110040e-002,1.0681731e-002,2.5121337e-002,1.0791851e-002,-1.6638584e-003,-1.9711647e-002,-2.0991802e-002,-1.4549728e-002,3.4275141e-002,-2.4804739e-002,-2.9292168e-002,1.4467137e-002,2.1074394e-002,1.1115331e-002,3.2812597e-003,1.4191835e-002,1.5912475e-002,-5.2436488e-004,4.9898546e-004,-5.6470013e-001,1.5494359e-003,7.3643373e-003,-3.3586882e-002,1.2526256e-003,3.0255726e-002,1.1218570e-002,-4.2736385e-004,-4.3194935e-002,-8.7236427e-004,-4.7420827e-003,7.2473339e-003,1.1865530e-002,-9.8489411e-003,-1.0998328e-002,-2.8658973e-002,3.8783215e-003,-2.4873566e-002,-7.8185862e-003,-1.6986154e-0

In [5]:

using Azure;
using Azure.AI.OpenAI;
using Azure.Identity;
using OpenAI.Embeddings;
using Newtonsoft.Json;
using Newtonsoft.Json.Linq;
using System.Net.Http;
using System.Collections.ObjectModel;
using Microsoft.Data.SqlClient;


In [6]:
var openAIClient = new AzureOpenAIClient(
    new Uri("https://savranweb.openai.azure.com/"),
    new AzureKeyCredential("3Qg3P8Fw37qaN3ZtG2JJapZZVjBFTKvOIDaIYXoL5a5fduu3wz2XJQQJ99BDACYeBjFXJ3w3AAABACOGPUlQ"));
var aiclient = openAIClient.GetEmbeddingClient("embedding");

In [7]:
ReadOnlyMemory<float> GenerateVector(string text)
{    
    OpenAIEmbedding newembedding = aiclient.GenerateEmbedding(text);
    return newembedding.ToFloats();
}

In [8]:
public class Post    
{    
    public int PostId { get; set; }
    public string PostBody { get; set; }
    public string Title { get; set; }
    public int ViewCount { get; set; }
    public int OwnerUserId { get; set; }
    public string PostType { get; set; }
    public int Score { get; set; }
    public float[] bodyvector {get;set;}
}

In [ ]:
var json = await new HttpClient().GetStringAsync("https://raw.githubusercontent.com/hsavran/Presentations/refs/heads/main/stackoverflow.json");
var postList = JsonConvert.DeserializeObject<List<Post>>(json);
postList.Count.Display();
var inserttable = new System.Data.DataTable("Posts");
inserttable.Columns.Add("PostId", typeof(int));
inserttable.Columns.Add("Title", typeof(string));
inserttable.Columns.Add("PostBody", typeof(string));
inserttable.Columns.Add("ViewCount", typeof(int));
inserttable.Columns.Add("PostType", typeof(string));
inserttable.Columns.Add("OwnerUserId", typeof(int));
inserttable.Columns.Add("Score", typeof(int));
inserttable.Columns.Add("bodyvector", typeof(string));

foreach (var post in postList.Where(p => p.PostType == "Question" && p.Title.Contains("SQL")).Take(1000))
{   
    var newRow = inserttable.NewRow();
    newRow["PostId"] = post.PostId;
    newRow["PostBody"] = post.PostBody;
    newRow["Title"] = post.Title;
    newRow["ViewCount"] = post.ViewCount;
    newRow["OwnerUserId"] = post.OwnerUserId;
    newRow["PostType"] = post.PostType;
    newRow["Score"] = post.Score;
    var temp = GenerateVector(post.PostBody).ToArray();
    var json = JsonConvert.SerializeObject(temp);
    newRow["bodyvector"] = json;
    //JsonConvert.SerializeObject(string.Join(",", temp));
    inserttable.Rows.Add(newRow);
}
//inserttable.Rows.Count.Display();
//inserttable.Rows[0].Display();
using (var connection = new Microsoft.Data.SqlClient.SqlConnection("Server=tcp:hsavransql1.database.windows.net,1433;Initial Catalog=azuredb1;Persist Security Info=False;User ID=hsavran;Password=HasanSavran90;MultipleActiveResultSets=False;Encrypt=True;TrustServerCertificate=False;Connection Timeout=30;"))
{
    connection.Open();
    using (var transaction = connection.BeginTransaction())
    {
        using (var bulkCopy = new Microsoft.Data.SqlClient.SqlBulkCopy(connection, Microsoft.Data.SqlClient.SqlBulkCopyOptions.Default, transaction))
        {
            bulkCopy.DestinationTableName = "Posts";
            bulkCopy.BatchSize = 1000;
            bulkCopy.WriteToServer(inserttable);
        }
        transaction.Commit();
    }
}

9264

134

System.Data.DataRow RowError RowState Added Table Posts CaseSensitive False IsInitialized True RemotingFormat Xml ChildRelations (empty) Columns index value 0 PostId AllowDBNull True AutoIncrement False AutoIncrementSeed 0 AutoIncrementStep 1 Caption PostId ColumnName PostId Prefix DataType System.Int32 DateTimeMode UnspecifiedLocal DefaultValue Expression ExtendedProperties (empty) MaxLength -1 Namespace Ordinal 0 ReadOnly False Table Posts CaseSensitive False IsInitialized True RemotingFormat Xml ChildRelations (empty) Columns index value 0 PostId 1 Title 2 PostBody 3 ViewCount 4 PostType 5 OwnerUserId 6 Score 7 bodyvector Constraints (empty) DataSet <null> DefaultView index value 0 System.Data.DataRowView 1 System.Data.DataRowView 2 System.Data.DataRowView 3 System.Data.DataRowView 4 System.Data.DataRowView 5 System.Data.DataRowView 6 System.Data.DataRowView 7 System.Data.DataRowView 8 System.Data.DataRowView 9 System.Data.DataRowView 10 System.Data.DataRowView 11 System.Data.DataRowView 12 System.Data.DataRowView 13 System.Data.DataRowView 14 System.Data.DataRowView 15 System.Data.DataRowView 16 System.Data.DataRowView 17 System.Data.DataRowView 18 System.Data.DataRowView 19 System.Data.DataRowView (114 more) DisplayExpression ExtendedProperties (empty) HasErrors False Locale en Parent LCID 9 KeyboardLayoutId 9 Name en IetfLanguageTag en DisplayName English NativeName English EnglishName English TwoLetterISOLanguageName en ThreeLetterISOLanguageName eng ThreeLetterWindowsLanguageName ENU CompareInfo CompareInfo - en TextInfo TextInfo - en IsNeutralCulture True CultureTypes NeutralCultures, InstalledWin32Cultures NumberFormat System.Globalization.NumberFormatInfo DateTimeFormat System.Globalization.DateTimeFormatInfo Calendar System.Globalization.GregorianCalendar OptionalCalendars [ System.Globalization.GregorianCalendar ] UseUserOverride True IsReadOnly False MinimumCapacity 50 ParentRelations (empty) PrimaryKey (empty) Rows index value 0 System.Data.DataRow 1 System.Data.DataRow 2 System.Data.DataRow 3 System.Data.DataRow 4 System.Data.DataRow 5 System.Data.DataRow 6 System.Data.DataRow 7 System.Data.DataRow 8 System.Data.DataRow 9 System.Data.DataRow 10 System.Data.DataRow 11 System.Data.DataRow 12 System.Data.DataRow 13 System.Data.DataRow 14 System.Data.DataRow 15 System.Data.DataRow 16 System.Data.DataRow 17 System.Data.DataRow 18 System.Data.DataRow 19 System.Data.DataRow (114 more) TableName Posts Namespace Prefix Site <null> Container <null> DesignMode False Unique False ColumnMapping Element Site <null> Container <null> DesignMode False 1 Title AllowDBNull True AutoIncrement False AutoIncrementSeed 0 AutoIncrementStep 1 Caption Title ColumnName Title Prefix DataType System.String DateTimeMode UnspecifiedLocal DefaultValue Expression ExtendedProperties (empty) MaxLength -1 Namespace Ordinal 1 ReadOnly False Table Posts CaseSensitive False IsInitialized True RemotingFormat Xml ChildRelations (empty) Columns index value 0 PostId 1 Title 2 PostBody 3 ViewCount 4 PostType 5 OwnerUserId 6 Score 7 bodyvector Constraints (empty) DataSet <null> DefaultView index value 0 System.Data.DataRowView 1 System.Data.DataRowView 2 System.Data.DataRowView 3 System.Data.DataRowView 4 System.Data.DataRowView 5 System.Data.DataRowView 6 System.Data.DataRowView 7 System.Data.DataRowView 8 System.Data.DataRowView 9 System.Data.DataRowView 10 System.Data.DataRowView 11 System.Data.DataRowView 12 System.Data.DataRowView 13 System.Data.DataRowView 14 System.Data.DataRowView 15 System.Data.DataRowView 16 System.Data.DataRowView 17 System.Data.DataRowView 18 System.Data.DataRowView 19 System.Data.DataRowView (114 more) DisplayExpression ExtendedProperties (empty) HasErrors False Locale en Parent LCID 9 KeyboardLayoutId 9 Name en IetfLanguageTag en DisplayName English NativeName English EnglishName English TwoLetterISOLanguageName en ThreeLetterISOLanguageName eng ThreeLetterWindowsLanguageName ENU CompareInfo CompareInfo - en TextInfo TextInf

In [49]:
var vectortosearch = GenerateVector("What are the most common questions about relational databases").ToArray();
var temp = JsonConvert.SerializeObject(vectortosearch);
// run the query to get the top 10 most similar posts
var cmd = @"
DECLARE @v AS VECTOR(1536);
--DECLARE @temp AS NVARCHAR(MAX);
SET @v = (SELECT CAST(@temp AS VECTOR(1536)))
SELECT TOP(10)   
  title,
  VECTOR_DISTANCE('cosine', @v, bodyvector) AS distance
FROM 
  Posts
ORDER BY
  distance desc";
using (var connection = new Microsoft.Data.SqlClient.SqlConnection("Server=tcp:hsavransql1.database.windows.net,1433;Initial Catalog=azuredb1;Persist Security Info=False;User ID=hsavran;Password=HasanSavran90;MultipleActiveResultSets=False;Encrypt=True;TrustServerCertificate=False;Connection Timeout=30;"))
{
    connection.Open();
    using (var command = new Microsoft.Data.SqlClient.SqlCommand(cmd, connection))
    {
        command.Parameters.AddWithValue("@temp", temp);
        using (var reader = command.ExecuteReader())
        {
            while (reader.Read())
            {
                Console.WriteLine($"Title: {reader["title"]}, Distance: {reader["distance"].ToString()}");               
              
            }
        }
    }
}

Title: What is best practice for FTP from a SQL Server 2005 stored procedure?, Distance: 0.3161234022465138
Title: Decoding T-SQL CAST in C#/VB.NET, Distance: 0.31561580662175304
Title: Using Parameters in MS Reporting Services (SQL Server 2008) against an ODBC data source, Distance: 0.3117094200667455
Title: SQL 2005 Reporting Services custom report item (CRI) - what are the limits?, Distance: 0.3108764845457265
Title: What is the best method for checking if a file exists from a SQL Server 2005 stored procedure?, Distance: 0.30944626746173876
Title: Best method for varchar date validation in Sybase (T-SQL)?, Distance: 0.3079388520235
Title: Using cached credentials to connect to SQL 2005 across a domain boundary, Distance: 0.304684993515298
Title: Conditional Visibility and Page Breaks with SQL Server 2005 Reporting Services, Distance: 0.3034064229479754
Title: Oracle SQL Developer not responsive when trying to view tables (or suggest an Oracle Mac client), Distance: 0.303236342609665